# 20. プレフィックスなしの埋め込み実験

## 背景

e5-largeモデルでは、以下のプレフィックスを使い分けることが推奨されている：
- ドキュメント: `passage: {text}`
- クエリ: `query: {text}`

しかし、LSH（SimHash）との相性問題から、**プレフィックスを使わない**埋め込みの効果を検証する。

## 仮説

プレフィックスを使わない場合、ドキュメントとクエリが同じ空間にマッピングされるため、
LSHのハミング距離がコサイン類似度とより強く相関する可能性がある。

---

# Phase 1: 小規模実験（1000件）

`scripts/no_prefix_experiment.py` を実行した結果。

## 実験設定

- **データ**: 1000件（各データセットから250件）
- **候補数**: 50, 100, 200件（5-20%の候補を選択）
- **クエリ**: 25件

### 検証パターン

| パターン | ドキュメント埋め込み | クエリ埋め込み |
|----------|----------------------|----------------|
| 現行 | passage: {text} | query: {text} |
| A | passage: {text} | passage: {text} |
| B | passage: {text} | {text} (なし) |
| C | {text} (なし) | query: {text} |
| D | {text} (なし) | passage: {text} |
| **E** | **{text} (なし)** | **{text} (なし)** |

## 結果: Recall@10（候補100件 / 1000件中 = 90%削減）

| パターン | Recall | 現行比 |
|----------|--------|--------|
| Doc=passage, Query=query (現行) | **36.8%** | - |
| Doc=passage, Query=passage | **83.6%** | +46.8pt |
| Doc=passage, Query=none | 57.2% | +20.4pt |
| Doc=none, Query=query | 53.2% | +16.4pt |
| Doc=none, Query=passage | 57.6% | +20.8pt |
| **Doc=none, Query=none** | **68.0%** | **+31.2pt** |

## クエリタイプ別（候補100件）

| パターン | JA短文 | JA曖昧 | EN短文 | EN曖昧 |
|----------|--------|--------|--------|--------|
| Doc=passage, Query=query (現行) | 44% | 38% | 24% | 34% |
| Doc=none, Query=none | **68%** | **62%** | **72%** | **70%** |

**英語クエリの改善が特に顕著:**
- EN短文: 24% → 72%（+48pt）
- EN曖昧: 34% → 70%（+36pt）

## Phase 1 考察

### 発見事項

1. **プレフィックスなし（Doc=none, Query=none）は現行から+31.2%改善**
   - 現行: 36.8% → プレフィックスなし: 68.0%

2. **最良は Doc=passage, Query=passage（83.6%）**
   - ただしこれはe5の非対称検索を完全に放棄する設定

3. **プレフィックスなしは現実的な妥協点**
   - e5の設計思想からは外れるが、LSHとの相性は大幅に改善
   - 英語クエリで特に効果的

### 理論的な解釈

e5モデルの `query:` / `passage:` プレフィックスは、クエリとドキュメントを**意図的に異なる空間**にマッピングしている。
これにより、コサイン類似度での検索精度が向上するが、**ハミング距離との相関が崩れる**。

プレフィックスなしの場合、クエリとドキュメントが**同じ空間**にマッピングされるため、
ハミング距離がコサイン類似度をより正確に近似できる。

---

# Phase 2: 大規模検証（40万件）

`scripts/create_no_prefix_index.py` で40万件の埋め込みを再生成し、検証を行う。

In [ ]:
import sys
sys.path.insert(0, '..')

import numpy as np
from numpy.linalg import norm
import pandas as pd
import duckdb
from tqdm import tqdm
from sentence_transformers import SentenceTransformer

from src.lsh import SimHashGenerator, hamming_distance

In [ ]:
# DuckDBに接続
con = duckdb.connect('../data/experiment_400k.duckdb', read_only=True)

# テーブル確認
tables = con.execute("SHOW TABLES").fetchall()
print('テーブル一覧:')
for table in tables:
    print(f'  - {table[0]}')

In [ ]:
# プレフィックスなしの埋め込みを読み込み
print('データ読み込み中...')
datasets = ['body_en', 'body_ja', 'titles_en', 'titles_ja']
embeddings_no_prefix = {}
embeddings_with_prefix = {}

for dataset in tqdm(datasets, desc='データセット'):
    # プレフィックスなし
    df_no = con.execute(f"""
        SELECT embedding FROM documents_no_prefix
        WHERE dataset = '{dataset}'
        ORDER BY id
    """).fetchdf()
    embeddings_no_prefix[dataset] = np.array(df_no['embedding'].tolist(), dtype=np.float32)
    
    # プレフィックスあり（既存）
    df_with = con.execute(f"""
        SELECT embedding FROM documents
        WHERE dataset = '{dataset}'
        ORDER BY id
    """).fetchdf()
    embeddings_with_prefix[dataset] = np.array(df_with['embedding'].tolist(), dtype=np.float32)

print('完了')

In [ ]:
# 全データを統合
all_embs_no_prefix = np.vstack([embeddings_no_prefix[d] for d in datasets])
all_embs_with_prefix = np.vstack([embeddings_with_prefix[d] for d in datasets])

print(f'プレフィックスなし: {all_embs_no_prefix.shape}')
print(f'プレフィックスあり: {all_embs_with_prefix.shape}')

In [ ]:
# E5モデルを読み込み
print('E5モデルを読み込み中...')
model = SentenceTransformer('intfloat/multilingual-e5-large')
print('完了')

In [ ]:
# 検索クエリ
search_queries = [
    ('東京', 'ja', 'short'),
    ('人工知能', 'ja', 'short'),
    ('日本の歴史', 'ja', 'short'),
    ('プログラミング', 'ja', 'short'),
    ('音楽', 'ja', 'short'),
    ('環境問題', 'ja', 'short'),
    ('宇宙探査', 'ja', 'short'),
    ('経済学', 'ja', 'short'),
    ('医療技術', 'ja', 'short'),
    ('文学作品', 'ja', 'short'),
    ('最近話題になっている技術革新について知りたいのですが', 'ja', 'ambiguous'),
    ('日本の伝統的な文化や芸術に関する情報', 'ja', 'ambiguous'),
    ('環境に優しい持続可能な社会を実現するための取り組み', 'ja', 'ambiguous'),
    ('健康的な生活を送るために必要なこと', 'ja', 'ambiguous'),
    ('世界の政治情勢や国際関係についての最新動向', 'ja', 'ambiguous'),
    ('Tokyo', 'en', 'short'),
    ('Artificial intelligence', 'en', 'short'),
    ('World history', 'en', 'short'),
    ('Programming', 'en', 'short'),
    ('Climate change', 'en', 'short'),
    ('Recent technological innovations and developments', 'en', 'ambiguous'),
    ('Traditional culture and arts from around the world', 'en', 'ambiguous'),
    ('Sustainable approaches to environmental protection', 'en', 'ambiguous'),
    ('Latest developments in space exploration', 'en', 'ambiguous'),
    ('Key factors for business success and entrepreneurship', 'en', 'ambiguous'),
]

query_texts = [q[0] for q in search_queries]
print(f'検索クエリ数: {len(query_texts)}')

In [ ]:
# クエリ埋め込みを生成
print('クエリ埋め込みを生成中...')

# プレフィックスなし
query_embs_none = model.encode(query_texts, normalize_embeddings=False).astype(np.float32)

# query:プレフィックス
query_embs_query = model.encode(
    [f'query: {t}' for t in query_texts],
    normalize_embeddings=False
).astype(np.float32)

print('完了')

In [ ]:
# 超平面を生成（各パターン用）
def generate_hyperplanes(embeddings, num_hyperplanes=128, seed=42):
    rng = np.random.default_rng(seed)
    sample_indices = rng.choice(len(embeddings), min(300, len(embeddings)), replace=False)
    sample_embs = embeddings[sample_indices]
    
    hyperplanes = []
    for _ in range(num_hyperplanes):
        i, j = rng.choice(len(sample_embs), 2, replace=False)
        diff = sample_embs[i] - sample_embs[j]
        diff = diff / np.linalg.norm(diff)
        hyperplanes.append(diff)
    
    return np.array(hyperplanes, dtype=np.float32)

# プレフィックスなし用の超平面
hp_no_prefix = generate_hyperplanes(all_embs_no_prefix)

# プレフィックスあり用の超平面
hp_with_prefix = generate_hyperplanes(all_embs_with_prefix)

print(f'超平面形状: {hp_no_prefix.shape}')

In [ ]:
# 評価関数
def evaluate_pattern(
    doc_embeddings: np.ndarray,
    query_embeddings: np.ndarray,
    hyperplanes: np.ndarray,
    search_queries: list,
    candidate_limits: list,
    top_k: int = 10
) -> pd.DataFrame:
    """
    パターンを評価してDataFrameを返す
    """
    gen = SimHashGenerator(dim=1024, hash_bits=128, seed=0, strategy='random')
    gen.hyperplanes = hyperplanes
    
    doc_hashes = gen.hash_batch(doc_embeddings)
    
    results = []
    
    for i, (query_text, lang, query_type) in enumerate(search_queries):
        query_emb = query_embeddings[i]
        query_hash = gen.hash_batch(query_emb.reshape(1, -1))[0]
        
        # Ground Truth
        cos_sims = (doc_embeddings @ query_emb) / (norm(doc_embeddings, axis=1) * norm(query_emb))
        gt_indices = set(np.argsort(cos_sims)[::-1][:top_k])
        
        # ハミング距離
        distances = [(j, hamming_distance(h, query_hash)) for j, h in enumerate(doc_hashes)]
        distances.sort(key=lambda x: x[1])
        
        for limit in candidate_limits:
            candidates = set(idx for idx, _ in distances[:limit])
            recall = len(gt_indices & candidates) / top_k
            
            results.append({
                'query': query_text,
                'lang': lang,
                'query_type': query_type,
                'candidate_limit': limit,
                'recall': recall
            })
    
    return pd.DataFrame(results)

In [ ]:
# 評価実行
print('評価を実行中...')
candidate_limits = [1000, 2000, 5000, 10000]

# パターン1: 現行（Doc=passage, Query=query）
print('  パターン1: Doc=passage, Query=query...')
df_current = evaluate_pattern(
    all_embs_with_prefix, query_embs_query, hp_with_prefix,
    search_queries, candidate_limits
)
df_current['pattern'] = 'Doc=passage, Query=query'

# パターン2: プレフィックスなし（Doc=none, Query=none）
print('  パターン2: Doc=none, Query=none...')
df_no_prefix = evaluate_pattern(
    all_embs_no_prefix, query_embs_none, hp_no_prefix,
    search_queries, candidate_limits
)
df_no_prefix['pattern'] = 'Doc=none, Query=none'

# パターン3: Doc=none, Query=query（ハイブリッド）
print('  パターン3: Doc=none, Query=query...')
df_hybrid = evaluate_pattern(
    all_embs_no_prefix, query_embs_query, hp_no_prefix,
    search_queries, candidate_limits
)
df_hybrid['pattern'] = 'Doc=none, Query=query'

# 結合
df_results = pd.concat([df_current, df_no_prefix, df_hybrid], ignore_index=True)
print('完了')

In [ ]:
# 結果表示
print('=' * 90)
print('結果: 40万件でのRecall@10比較')
print('=' * 90)

pivot = df_results.groupby(['pattern', 'candidate_limit'])['recall'].mean().unstack()

print(f'\n{"パターン":>30} | {"1000件":>10} | {"2000件":>10} | {"5000件":>10} | {"10000件":>10}')
print('-' * 80)

for pattern in ['Doc=passage, Query=query', 'Doc=none, Query=none', 'Doc=none, Query=query']:
    if pattern in pivot.index:
        row = pivot.loc[pattern]
        print(f'{pattern:>30} | {row[1000]:>10.1%} | {row[2000]:>10.1%} | {row[5000]:>10.1%} | {row[10000]:>10.1%}')

In [ ]:
# クエリタイプ別（候補2000件）
print('\n' + '=' * 85)
print('クエリタイプ別 Recall@10（候補2000件）')
print('=' * 85)

subset = df_results[df_results['candidate_limit'] == 2000]

print(f'\n{"パターン":>30} | {"JA短文":>10} | {"JA曖昧":>10} | {"EN短文":>10} | {"EN曖昧":>10}')
print('-' * 85)

for pattern in ['Doc=passage, Query=query', 'Doc=none, Query=none', 'Doc=none, Query=query']:
    pattern_subset = subset[subset['pattern'] == pattern]
    
    ja_short = pattern_subset[(pattern_subset['lang'] == 'ja') & (pattern_subset['query_type'] == 'short')]['recall'].mean()
    ja_amb = pattern_subset[(pattern_subset['lang'] == 'ja') & (pattern_subset['query_type'] == 'ambiguous')]['recall'].mean()
    en_short = pattern_subset[(pattern_subset['lang'] == 'en') & (pattern_subset['query_type'] == 'short')]['recall'].mean()
    en_amb = pattern_subset[(pattern_subset['lang'] == 'en') & (pattern_subset['query_type'] == 'ambiguous')]['recall'].mean()
    
    print(f'{pattern:>30} | {ja_short:>10.1%} | {ja_amb:>10.1%} | {en_short:>10.1%} | {en_amb:>10.1%}')

In [ ]:
# 接続を閉じる
con.close()
print('完了')

---

# 結論

## Phase 1（小規模実験、1000件）

| パターン | Recall (100件/1000件中) |
|----------|-------------------------|
| Doc=passage, Query=query (現行) | 36.8% |
| Doc=none, Query=none | **68.0%** (+31.2pt) |

## Phase 2（大規模検証、40万件）

`scripts/evaluate_no_prefix_index.py` の結果：

### Recall@10（候補2000件）

| パターン | Recall | 現行比 |
|----------|--------|--------|
| Baseline (passage/query) | 32.3% | - |
| **No Prefix (none/none)** | **44.0%** | **+11.7pt** |
| Hybrid (none/query) | 40.3% | +8.0pt |
| Phase1 Best (passage/passage) | 69.3% | +37.0pt |

### 候補数別推移

| パターン | 500 | 1000 | 2000 | 5000 | 10000 | 20000 |
|----------|-----|------|------|------|-------|-------|
| Baseline (passage/query) | 17.0% | 24.3% | 32.3% | 45.0% | 56.0% | 66.0% |
| No Prefix (none/none) | 26.3% | 37.0% | 44.0% | 55.3% | 67.0% | 76.0% |
| Hybrid (none/query) | 25.0% | 33.0% | 40.3% | 52.7% | 64.7% | 72.0% |
| Phase1 Best (passage/passage) | 52.0% | 61.3% | 69.3% | 80.7% | 89.0% | 94.3% |

### クエリタイプ別（候補2000件）

| パターン | JA短文 | JA曖昧 | EN短文 | EN曖昧 |
|----------|--------|--------|--------|--------|
| Baseline (passage/query) | 34.0% | 23.0% | 58.0% | 22.0% |
| **No Prefix (none/none)** | **62.0%** | **33.0%** | 48.0% | 26.0% |
| Phase1 Best (passage/passage) | 81.0% | 63.0% | 74.0% | 54.0% |

### 改善度分析

**改善したクエリ（Top 5）:**
- 「プログラミング」: 30% → 90% (+60pt)
- 「経済学」: 20% → 80% (+60pt)
- 「東京」: 10% → 60% (+50pt)

**悪化したクエリ:**
- 「World history」: 50% → 0% (-50pt)
- 「子供の教育において...」: 70% → 10% (-60pt)

## 総合評価

1. **プレフィックスなしで11.7%の改善**（32.3% → 44.0%）
   - 日本語短文で特に効果的（+28pt）
   - 一部の英語クエリで悪化あり

2. **最良はpassage/passageパターン（69.3%）**
   - これはe5の非対称検索を放棄する設定
   - 実用上の問題がないならこれが最適

3. **プレフィックスなしは現実的な妥協点**
   - e5設計からは外れるが、LSH相性は改善
   - 日本語検索主体のシステムで有効

## 推奨事項

**LSH候補選択には以下のいずれかを推奨:**

1. **passage/passage**: Recall@10 = 69.3%（候補2000件）
   - e5の非対称設計を放棄するが、最高精度

2. **none/none**: Recall@10 = 44.0%（候補2000件）
   - 中程度の精度だが、日本語に強い

3. **候補数の増加**: 20000件まで増やせばbaselineでも66%達成
   - 計算コストとのトレードオフ

**結論**: SimHashとe5-largeの組み合わせには本質的な限界があり、
FAISS/Annoy/ScaNNなどの専用ライブラリへの移行も検討すべき。